In [ ]:
import pandas as pd

df = pd.read_csv('data/data4.tsv', delimiter = '\t')

df

In [ ]:
import numpy as np
from keras.preprocessing.image import load_img, img_to_array

imgs = np.array([ img_to_array(load_img(i)) for i in df['image'].values ])

imgs.shape

In [ ]:
import cv2

def label_img(ps, line_size = 6):
    shape = tuple(imgs.shape[1:-1]) + (1,)
    
    img = np.zeros(shape)
    
    for i in range(0, len(ps), 4):
        cv2.line(img, (ps[i], ps[i + 1]), (ps[i + 2], ps[i + 3]), 255, line_size)
    
    return img

In [ ]:
cols = ['x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4', 'x5', 'y5', 'x6', 'y6', 'x7', 'y7', 'x8', 'y8']

labels = np.array([label_img(d) / 255 for d in df[cols].values])

labels.shape

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

def imshow_label(index, s = 6.0):
    plt.rcParams['figure.figsize'] = (s, s)

    plt.subplot(1, 3, 1)
    plt.imshow(imgs[index].astype(int))

    plt.subplot(1, 3, 2)
    plt.imshow(labels[index].reshape(labels.shape[1:-1]), cmap = 'gray')

    img = cv2.addWeighted(imgs[index][:, :, 0].astype(int), 0.5, (labels[index] * 255).astype(int), 0.5, 0)
    
    plt.subplot(1, 3, 3)
    plt.imshow(img, cmap = 'gray')

    

In [ ]:
imshow_label(0)

In [ ]:
imshow_label(1)

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Dropout, UpSampling2D, Reshape
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPool2D
from keras.layers.normalization import BatchNormalization

input = Input(shape = imgs.shape[1:])

x = BatchNormalization()(input)

x = Conv2D(16, 3, padding = 'same', activation = 'relu')(x)
x = MaxPool2D()(x)

x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = Conv2D(32, 3, padding = 'same', activation = 'relu')(x)
x = Conv2D(32, 3, padding = 'same', activation = 'relu')(x)
x = MaxPool2D()(x)

x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = Conv2D(64, 3, padding = 'same', activation = 'relu')(x)
x = Conv2D(64, 3, padding = 'same', activation = 'relu')(x)
x = MaxPool2D()(x)

x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = Conv2D(128, 3, padding = 'same', activation = 'relu')(x)
x = Conv2D(128, 3, padding = 'same', activation = 'relu')(x)
x = MaxPool2D()(x)

x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = Conv2D(16, 1, padding = 'same', activation = 'relu')(x)

f = Flatten()
x = f(x)

print(f"{f.input_shape}, {f.output_shape}")

x = Dense(int(f.output_shape[-1] / 2))(x)
x = Dense(int(f.output_shape[-1] / 4))(x)

x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = Reshape(tuple(f.input_shape[1:-1]) + (4,))(x)

x = UpSampling2D()(x)
x = Conv2D(128, 3, padding = 'same', activation = 'relu')(x)
x = Conv2D(128, 3, padding = 'same', activation = 'relu')(x)

x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = UpSampling2D()(x)
x = Conv2D(64, 3, padding = 'same', activation = 'relu')(x)
x = Conv2D(64, 3, padding = 'same', activation = 'relu')(x)

x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = UpSampling2D()(x)
x = Conv2D(32, 3, padding = 'same', activation = 'relu')(x)
x = Conv2D(32, 3, padding = 'same', activation = 'relu')(x)

x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = UpSampling2D()(x)
x = Conv2D(16, 3, padding = 'same', activation = 'relu')(x)

x = BatchNormalization()(x)
x = Dropout(0.2)(x)

output = Conv2D(1, 1, activation = 'sigmoid')(x)

model = Model(inputs = input, outputs = output)
          
model.compile(loss = 'mae', optimizer = 'adam', metrics = ['mse'])

model.summary()

In [ ]:
hist = model.fit(imgs, labels, epochs = 50, batch_size = 10)
hist

In [ ]:
plt.rcParams['figure.figsize'] = (4.0, 4.0)
    
plt.plot(hist.history['loss'])

In [ ]:
model.save('model/cnn-4_0.h5')

In [ ]:
import cv2

def predict(i, s = 8.0):
    plt.rcParams['figure.figsize'] = (s, s)
    
    r = (labels[i] * 255).astype(int)
    p = (model.predict(np.array([imgs[i]]))[0] * 255).astype(int)
   
    img = imgs[i][:, :, 0].astype(int)
    
    img2 = cv2.addWeighted(img, 0.5, r, 0.5, 0)
    img3 = cv2.addWeighted(img, 0.5, p, 0.5, 0)
    
    plt.subplot(1, 4, 1)
    plt.imshow(img2, cmap = 'gray')

    plt.subplot(1, 4, 2)
    plt.imshow(img3, cmap = 'gray')

    plt.subplot(1, 4, 3)
    plt.imshow(p.reshape(p.shape[:-1]), cmap = 'gray')


In [ ]:
predict(0)

In [ ]:
predict(1)

In [ ]:
predict(2)

In [ ]:
predict(3)

In [ ]:
predict(4)